In [125]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time

def StartUp():
    GPIO.setmode(GPIO.BCM)
    PRESET = 25
    IO4 = 26
    GPIO.setup(PRESET,GPIO.OUT)
    GPIO.setup(IO4,GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
    print "Reset 25 - Low 1s"
    GPIO.output(PRESET,GPIO.LOW)
    time.sleep(3)
    print "Reset 25 - High 0.2s"
    GPIO.output(PRESET,GPIO.HIGH)
    time.sleep(0.2)

    spi = spidev.SpiDev()
    spi.open(0,1) # CS2 - FPGA, on CE1 = IO4
    spi.mode = 0b00
    print "spi.cshigh is " + str(spi.cshigh)
    print "spi mode is " + str(spi.mode)
    spi.max_speed_hz = 200000
    print "spi maxspeed is "+str(spi.max_speed_hz)+"hz"
    return spi

def TestSPI(spi,ncycles):
    i = 0
    while i < ncycles:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        time.sleep(0.5)
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
        time.sleep(0.5)  
        i = i+1
        
def ConfigSPI(spi):
    WriteFPGA(spi,0xE0,0x14) # set sEEPon
    WriteFPGA(spi,0xE1,0x00) # set sEEPoff MSB
    WriteFPGA(spi,0xE2,0xC8) # set sEEPoff LSB
    WriteFPGA(spi,0xE3,0x00) # set sEEDelayACQ sEEDelayACQ MSB
    WriteFPGA(spi,0xE4,0xBC) # set sEEDelayACQ EEDelayACQ LSB
    WriteFPGA(spi,0xE5,0x00) # set sEEPeriod MSB
    WriteFPGA(spi,0xE6,0xC8) # set sEEPeriod LSB
    WriteFPGA(spi,0xE7,0x01) # Period of one cycle MSB
    WriteFPGA(spi,0xE8,0x86) # Period of one cycle 15 to 8
    WriteFPGA(spi,0xE9,0xA0) # Period of one cycle LSB
    #WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
    WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
    WriteFPGA(spi,0xEC,0x11) # Voltage gain control: 0V to 1V
    WriteFPGA(spi,0xED,0x03) # Frequency of ADC acquisition / sEEADC_freq
    WriteFPGA(spi,0xEE,0xA0) # How many cycles in countinious mode
    print "Config FPGA done!"

def WriteFPGA(spi,adress,value):
    spi.xfer([0xAA] )
    spi.xfer([adress] )
    spi.xfer([value] )

In [126]:
int(0x0186A0)

100000

In [127]:
spi = StartUp()
ConfigSPI(spi)

Reset 25 - Low 1s
Reset 25 - High 0.2s
spi.cshigh is False
spi mode is 0
spi maxspeed is 200000hz
Config FPGA done!


In [128]:
#TestSPI(spi,3)

In [93]:
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

In [ ]:
WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
while 1:
    WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software (normally OK, 1 acq is 1ms)
    time.sleep(0.005) # sleep 5ms

In [123]:
WriteFPGA(spi,0xEE,0xBB) # How many cycles in countinious mode
WriteFPGA(spi,0xEB,0x00) # COntinuous
WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software
time.sleep(0.01) 
for i in range(1011):
    spi.xfer([0xBB] )
print spi.xfer([0x1])
print spi.xfer([0xBB])
time.sleep(0.001) 
print spi.xfer([0xBB])

[170]
[170]
[170]


In [97]:
print spi.xfer([0xA1])
print spi.xfer([0xA1])
time.sleep(0.001) 
print spi.xfer([0xA1])

[170]
[170]
[170]


In [62]:
ct = []
for i in range(10101):
    ct.append( spi.xfer([0x81] )[0]  ) 
last = 0
for j in range(len(ct)): 
    if not(ct[j] == last):
        print last,ct[j]
    last = ct[j]
    #print j

0 170


In [96]:
spi.readbytes(100)

[170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170,
 170]

In [86]:
spi.close() 